In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = True
%load_ext sql

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import pickle
import datetime

In [ ]:
# データベースに接続
conn = sqlite3.connect('tataki.sqlite3')
curs = conn.cursor()

In [ ]:
df=pd.read_sql_query('SELECT * FROM tataki', conn)
# print(df)
print(len(df))

In [ ]:
# カーソルとコネクションをクローズする
curs.close()
conn.close()

In [ ]:
# // array(1次元)をtargetLengsの長さにする
def getArrayLengsChange(array, targetLengs):
    targetArray = [0] * targetLengs
    if len(array) == targetLengs:
        return array
    # 長さ100を長さ3にするとしたら、25, 50, 75番目を選んでる
    elif len(array)> targetLengs:
        targetIndexTips = len(array) / (targetLengs + 1)
        k = 1
        targetIndex = int(targetIndexTips * k)
        for i in range(targetLengs):
            targetArray[i] =array[targetIndex]
            k += 1
            targetIndex = int(targetIndexTips * k)
    # 長さ13から長さ16への変換でデータを3個追加するとしたら、13を(3+1)等分した箇所にデータを追加する
    elif len(array) < targetLengs:
        diff = targetLengs - len(array)
        targetIndexTips = targetLengs / (diff + 1)
        k = 1
        targetIndex = int(targetIndexTips * k)
        j = 0
        for i in range(targetLengs):
            if i == targetIndex:
                if j == 0:
                    targetArray[i] = array[j]
                else:
                    targetArray[i] = (array[j-1] + array[j]) / 2
                k += 1
                targetIndex = int(targetIndexTips * k);
            else:
                targetArray[i] = array[j]
                j += 1
    return targetArray

In [ ]:
# numpyにして高速化

from sklearn import preprocessing

leng = 16

np_zurasi = np.array([-7, -6, -5, -4, -3, -2, -1, 0, 1, 2])
zurasi_bai = np_zurasi.shape[0]

noise_bai = 5 #ノイズでn倍

dlen = 6*leng #ノイズデータのデータ長
mean = 0.0    #ノイズの平均値
std  = 0.1    #ノイズの分散


np_ax = df['AX'].to_numpy()
np_ay = df['AY'].to_numpy()
np_az = df['AZ'].to_numpy()
np_rx = df['RX'].to_numpy()
np_ry = df['RY'].to_numpy()
np_rz = df['RZ'].to_numpy()

np_kindsSumaho = df['kindsSumaho'].to_numpy()
np_orientation = df['orientation'].to_numpy()
np_kindsTataki = df['kindsTataki'].to_numpy()

# n個を持った配列を用意しておくと早くなる(しておかないとめっちゃ遅い)
data_num = df.shape[0] * zurasi_bai * noise_bai
mynps = np.empty((data_num, leng, 6))

print('df.shape[0]:', df.shape[0])
data_num_count = 0
for idx in range(df.shape[0]):
# for idx in range(1):
    if(np_kindsTataki[idx] != np_kindsTataki[idx - 1]):
        print(idx, np_kindsSumaho[idx], np_orientation[idx], np_kindsTataki[idx])
    
    # 長さを変える
    ax = np_ax[idx].split(',')
    ax = list(map(lambda x: float(x), ax))
    ax = getArrayLengsChange(ax, leng)
#     ax = preprocessing.scale(ax)
    
    ay = np_ay[idx].split(',')
    ay = list(map(lambda x: float(x), ay))
    ay = getArrayLengsChange(ay, leng)
#     ay = preprocessing.scale(ay)
    
    az = np_az[idx].split(',')
    az = list(map(lambda x: float(x), az))
    az = getArrayLengsChange(az, leng)
#     az = preprocessing.scale(az)
    
    rx = np_rx[idx].split(',')
    rx = list(map(lambda x: float(x), rx))
    rx = getArrayLengsChange(rx, leng)
#     rx = preprocessing.scale(rx)
    
    ry = np_ry[idx].split(',')
    ry = list(map(lambda x: float(x), ry))
    ry = getArrayLengsChange(ry, leng)
#     ry = preprocessing.scale(ry)
    
    rz = np_rz[idx].split(',')
    rz = list(map(lambda x: float(x), rz))
    rz = getArrayLengsChange(rz, leng)
#     rz = preprocessing.scale(rz)
    
    # 加速度と角速度ごとに標準化
    ax = np.array(ax)
    ay = np.array(ay)
    az = np.array(az)
    rx = np.array(rx)
    ry = np.array(ry)
    rz = np.array(rz)
    
    ax = ax.reshape(-1, 1)
    ay = ay.reshape(-1, 1)
    az = az.reshape(-1, 1)
    rx = rx.reshape(-1, 1)
    ry = ry.reshape(-1, 1)
    rz = rz.reshape(-1, 1)
    
    a_ = np.concatenate([ax, ay, az], 1)
    r_ = np.concatenate([rx, ry, rz], 1)
    
    
    a_shape = a_.shape
    r_shape = r_.shape
    
    a_ = a_.reshape(-1, 1)
    r_ = r_.reshape(-1, 1)
    
    a_ = preprocessing.scale(a_)
    r_ = preprocessing.scale(r_)
    
    a_ = a_.reshape(a_shape)
    r_ = r_.reshape(r_shape)
    
    # 結合
    np_ = np.concatenate([a_, r_], 1)
    
    # 前後にずらす
    for roll_len in np_zurasi:
        if roll_len > 0:
            np_roll = np_[:-roll_len]
            np_roll = np.concatenate([np.zeros([roll_len, 6]), np_roll], axis=0)
        elif roll_len == 0:
            np_roll = np_
        else:
            np_roll = np_[-roll_len:]
            np_roll = np.concatenate([np_roll, np.zeros([abs(roll_len), 6])], axis=0)
            
        # ノイズを重ねる
        for i in range(noise_bai):
            np_noise = np.random.normal(mean, std, dlen).reshape([-1, 6])
            np_noise = np_roll + np_noise

            # numpyに追加
            mynps[data_num_count] = np_noise
            
            # 表示
#             columns = ['AX', 'AY', 'AZ', 'RX', 'RY', 'RZ']
#             title = np_kindsSumaho[idx] + ' ' + np_orientation[idx] + ' ' + np_kindsTataki[idx]
#             tmp_df = pd.DataFrame(data=np_noise, columns=columns, dtype='float')
#             tmp_df.plot(title=title, style='.-')
            
            # index
            data_num_count += 1
    
# センサデータ
print(mynps.shape)


In [ ]:
kindsSumaho_list = []
orientation_list = []
kindsTataki_list = []

n_bai = zurasi_bai * noise_bai
    
for index, df_ in df.iterrows():
# for df_ in df.itertuples():
    print(index, df_['kindsSumaho'], df_['orientation'], df_['kindsTataki'])
    
    for i in range(n_bai):
        # listに追加
        if df_['kindsSumaho'] == 'SE3':
            kindsSumaho_list.append(0)
        elif df_['kindsSumaho'] == 'Nexus9':
            kindsSumaho_list.append(1)
        elif df_['kindsSumaho'] == 'FireHD10':
            kindsSumaho_list.append(2)

        if df_['orientation'] == 'vertical':
            orientation_list.append(0)
        elif df_['orientation'] == 'horizontal':
            orientation_list.append(1)

        if df_['kindsTataki'] == 'tatakiTopRight':
            kindsTataki_list.append(0)
        elif df_['kindsTataki'] == 'tatakiBottomRight':
            kindsTataki_list.append(1)
        elif df_['kindsTataki'] == 'tatakiTopLeft':
            kindsTataki_list.append(2)
        elif df_['kindsTataki'] == 'tatakiBottomLeft':
            kindsTataki_list.append(3)
        elif df_['kindsTataki'] == 'tatakiMisDetectioin':
            kindsTataki_list.append(4)
        elif df_['kindsTataki'] == 'tatakiHorizontallyRight':
            kindsTataki_list.append(5)
        elif df_['kindsTataki'] == 'tatakiHorizontallyLeft':
            kindsTataki_list.append(6)
        elif df_['kindsTataki'] == 'tatakiHorizontallyMisDetectioin':
            kindsTataki_list.append(7)

        
# 機種、向き、叩きの種類のデータ
kindsSumaho_list = np.array(kindsSumaho_list)
orientation_list = np.array(orientation_list)
kindsTataki_list = np.array(kindsTataki_list)

print(kindsSumaho_list.shape)
print(orientation_list.shape)
print(kindsTataki_list.shape)

In [ ]:
from keras.utils.np_utils import to_categorical

kindsSumaho_onehot = to_categorical(kindsSumaho_list)
orientation_onehot = to_categorical(orientation_list)
kindsTataki_onehot = to_categorical(kindsTataki_list)

kindsSumaho_onehot.shape
orientation_onehot.shape
kindsTataki_onehot.shape

In [ ]:
tatakiAllDatas = {
    'tataki_data': mynps,
    'orientation_onehot': orientation_onehot,
    'kindsSumaho_onehot': kindsSumaho_onehot,
    'kindsTataki_onehot': kindsTataki_onehot
}

print(tatakiAllDatas['tataki_data'].shape)
print(tatakiAllDatas['orientation_onehot'].shape)
print(tatakiAllDatas['kindsSumaho_onehot'].shape)
print(tatakiAllDatas['kindsTataki_onehot'].shape)

In [ ]:
import pickle
import datetime

dt_now = datetime.datetime.now()
dt_now = dt_now.strftime('%Y-%m-%d-%H-%M-%S')
file_name = 'tatakiAllDatas' + dt_now + '.pickle'
print(file_name)

with open(file_name, 'wb') as web:
    pickle.dump(tatakiAllDatas , web)

In [ ]:
test_size = 100*24*n_bai
print(test_size)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, muki_train, muki_test, kishu_train, kishu_test = train_test_split(tatakiAllDatas['tataki_data'],
                                                                                                    tatakiAllDatas['kindsTataki_onehot'],
                                                                                                    tatakiAllDatas['orientation_onehot'],
                                                                                                    tatakiAllDatas['kindsSumaho_onehot'],
                                                                                                    test_size=test_size,
                                                                                                    shuffle=True,
                                                                                                    random_state=0,
                                                                                                    stratify=tatakiAllDatas['kindsTataki_onehot']
                                                                                                   )


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(muki_train.shape)
print(kishu_train.shape)

print(x_test.shape)
print(y_test.shape)
print(muki_test.shape)
print(kishu_test.shape)

In [ ]:
import tensorflow as tf

inputs   = tf.keras.layers.Input(shape=x_train.shape[1:], name='inputs_waveform')

inputs_muki   = tf.keras.layers.Input(shape=muki_train.shape[1:], name='inputs_orientation')
# relu_muki     = tf.keras.layers.Dense(64, activation='tanh', name='relu_orientation')(inputs_muki)
relu_muki     = tf.keras.layers.Dense(8, activation='tanh', name='relu_orientation')(inputs_muki)

inputs_kishu   = tf.keras.layers.Input(shape=kishu_train.shape[1:], name='inputs_kinds')
# relu_kishu     = tf.keras.layers.Dense(64, activation='tanh', name='relu_kinds')(inputs_kishu)
relu_kishu     = tf.keras.layers.Dense(8, activation='tanh', name='relu_kinds')(inputs_kishu)

# gru1     = tf.keras.layers.GRU(64, return_sequences=False, name='gru1')(inputs)
# dropout1 = tf.keras.layers.Dropout(0.1, name='dropout1')(gru1)

lstm1    = tf.keras.layers.LSTM(64, return_sequences=False)(inputs)
dropout2 = tf.keras.layers.Dropout(0.1)(lstm1)

rnn1     = tf.keras.layers.SimpleRNN(64, return_sequences=False)(inputs)
dropout3 = tf.keras.layers.Dropout(0.1)(rnn1)

# relu1     = tf.keras.layers.SimpleRNN(64, return_sequences=False)(inputs)
# dropout4     = tf.keras.layers.Dropout(0.1)(xxx1)

# relu1    = tf.keras.layers.Dense(64, activation='relu', name='relu')(inputs)
# dropout4 = tf.keras.layers.Dropout(0.1)(relu1)

concatenate1 = tf.keras.layers.Concatenate()([dropout2, dropout3, relu_muki, relu_kishu])
# concatenate1 = tf.keras.layers.Concatenate()([dropout2, dropout3])

relu2    = tf.keras.layers.Dense(64, activation='relu', name='relu1')(concatenate1)
# # dropout5 = tf.keras.layers.Dropout(0.1)(relu2)
# dropout5 = tf.keras.layers.Dropout(0.2)(relu2)
# relu3    = tf.keras.layers.Dense(64, activation='relu', name='relu2')(dropout5)

# # concatenate2 = tf.keras.layers.Concatenate()([relu2, relu3])
# concatenate2 = tf.keras.layers.Concatenate()([dropout5, relu3])

# relu4    = tf.keras.layers.Dense(64, activation='relu', name='relu3')(concatenate2)
# dropout6 = tf.keras.layers.Dropout(0.1)(relu4)

# outputs  = tf.keras.layers.Dense( y_train.shape[-1] , activation='softmax', name='softmax')(dropout6)
# outputs  = tf.keras.layers.Dense( y_train.shape[-1] , activation='softmax', name='softmax')(dropout5)
outputs  = tf.keras.layers.Dense( y_train.shape[-1] , activation='softmax', name='softmax')(relu2)


# 入出力を定義します。
model    = tf.keras.models.Model(inputs=[inputs, inputs_muki, inputs_kishu], outputs=outputs)



# opt = tf.optimizers.Adam(lr=0.1)
# model.compile(optimizer=opt, loss='mse')
model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)
history = model.fit([x_train, muki_train, kishu_train], y_train,
                    validation_data = ([x_test, muki_test, kishu_test], y_test),
                    batch_size=32,
                    epochs=32,
#                     batch_size=64,
#                     epochs=16,
#                     batch_size=16,
#                     epochs=64,
                    verbose=1)

# print(model.predict([x_test, muki_test, kishu_test]))

In [ ]:
score = model.evaluate([x_test, muki_test, kishu_test], y_test)
print(score[0])
print(score[1])

In [ ]:
predict_prob = model.predict([x_test, muki_test, kishu_test])

predict_classes = np.argmax(predict_prob, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

labels = ['vertically/topRight', 'vertically/bottomRight', 'vertically/topLeft', 'vertically/bottomLeft', 'vertically/misDetectioin',
          'horizontally/topRight', 'horizontally/bottomRight', 'horizontally/misDetectioin']

y_true = np.argmax(y_test, axis=1)
y_pred = predict_classes
cm = confusion_matrix(y_true, y_pred)#, labels=labels) 

columns_labels = ["pred_" + str(l) for l in labels]
index_labels = ["act_" + str(l) for l in labels]
cm = pd.DataFrame(cm, columns=columns_labels, index=index_labels)

In [ ]:
plt.figure(figsize = (6, 6))
sns.heatmap(cm, square=True, cbar=True, annot=True, fmt='d', cmap='Blues')

In [ ]:
metrics = ['loss', 'accuracy']  # 使用する評価関数を指定
 
plt.figure(figsize=(10, 5))  # グラフを表示するスペースを用意
 
for i in range(len(metrics)):
 
    metric = metrics[i]
 
    plt.subplot(1, 2, i+1)  # figureを1×2のスペースに分け、i+1番目のスペースを使う
    plt.title(metric)  # グラフのタイトルを表示
    
    plt_train = history.history[metric]  # historyから訓練データの評価を取り出す
    plt_test = history.history['val_' + metric]  # historyからテストデータの評価を取り出す
    
    plt.plot(plt_train, label='training')  # 訓練データの評価をグラフにプロット
    plt.plot(plt_test, label='validation')  # テストデータの評価をグラフにプロット
    plt.legend()  # ラベルの表示
    
plt.show()  # グラフの表示

In [ ]:
dt_now = datetime.datetime.now()
dt_now = dt_now.strftime('%Y-%m-%d-%H-%M-%S')
my_model_path = 'tatakimodel' + dt_now + '.h5'
print(my_model_path)

model.save(my_model_path)

In [ ]:
import tensorflowjs as tfjs

dt_now = datetime.datetime.now()
dt_now = dt_now.strftime('%Y-%m-%d-%H-%M-%S')
tfjs_target_dir = './tfjs' + dt_now
print(tfjs_target_dir)

tfjs.converters.save_keras_model(model, tfjs_target_dir)

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(y_true, y_pred)

a

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)